# Using Parallel Computing for Macroeconomic Forecasting at the Federal Reserve Bank of New York

**Pearl Li** (@pearlzli) <br>
**Federal Reserve Bank of New York** (@FRBNY-DSGE)

June 21, 2017

## Disclaimer

This talk reflects the experience of the author and does not represent an endorsement by the Federal Reserve Bank of New York or the Federal Reserve System of any particular product or service. The views expressed in this talk are those of the authors and do not necessarily reflect the position of the Federal Reserve Bank of New York or the Federal Reserve System. Any errors or omissions are the responsibility of the authors.

## Outline

1. Overview of DSGE modeling
2. "The forecast step": objectives and challenges
3. Parallelizing the forecast code: <br>
   a. DistributedArrays.jl <br>
   b. `pmap` and blocking
4. Conclusion and next steps

## Overview of DSGE modeling

A DSGE (dynamic stochastic general equilibrium) model is a "micro-founded macro-model", used in both policy and academia for:

- Forecasting macroeconomic variables (GDP growth, inflation, interest rate, ...)
- Understanding the forces underlying past economic outcomes
- Analyzing the effect of monetary policy
- Investigating counterfactuals or alternative scenarios

Let

- $\theta$ be a vector of parameters (discount rate, capital share, ...)
- $s_t$ be a vector of latent states (output growth, natural rate of interest, ...), including expectations of future states and lags
- $\epsilon_t$ be a vector of exogenous shocks (productivity shock, ...)
- $\eta_t$ be a vector of expectational errors

We express the evolution of the system over time in terms of **equilibrium conditions** (derived from micro theory)

$$\Gamma_0(\theta) s_t = \Gamma_1(\theta) s_{t-1} + \Psi(\theta) \epsilon_t + \Pi(\theta) \eta_t + C(\theta)$$
$$\epsilon_t \sim N(0, Q(\theta))$$

which are solved to give the **transition equation**

$$s_t = T(\theta) s_{t-1} + R(\theta) \epsilon_t + C(\theta)$$

Let $y_t$ be a vector of observed variables (real GDP growth, core PCE inflation, ...). We map latent states $s_t$ to observables $y_t$ using the **measurement equation**

$$y_t = Z(\theta) s_t + D(\theta)$$

We are interested in

- Sampling from the posterior distribution $\mathbb{P}(\theta | y_{1:T})$ of the parameters $\theta$ (**estimation step**)
- Using the estimated parameter draws to forecast, compute impulse responses and shock decompositions, and more (**forecast step**)